In [ ]:
!pip install spacy
!pip install wordcloud

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
import numpy as np
import pandas as pd
import string
import spacy
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

## Emotion Mining

In [ ]:
afinn=pd.read_csv("Afinn.csv",sep=",",encoding="latin-1")
afinn

,word,value
0,abandon,-2
1,abandoned,-2
2,abandons,-2
3,abducted,-2
4,abduction,-2
...,...,...
2472,yucky,-2
2473,yummy,3
2474,zealot,-2
2475,zealots,-2


In [ ]:
afinn[1500:]

,word,value
1500,masterpieces,4
1501,matter,1
1502,matters,1
1503,mature,2
1504,meaningful,2
...,...,...
2472,yucky,-2
2473,yummy,3
2474,zealot,-2
2475,zealots,-2


In [ ]:
txt="   banana   "
print("of all fruit",txt,"is my favorite")

x=txt.strip()
print("of all fruits",x,"is my favorite")

of all fruit    banana    is my favorite
of all fruits banana is my favorite


In [ ]:
book=pd.read_csv("apple.txt",error_bad_lines=False)
book

book=[x.strip() for x in book.x]
book=[x for x in book if x]
book[0:10]

<ipython-input-12-04430f639521>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book=pd.read_csv("apple.txt",error_bad_lines=False)
Skipping line 6: expected 1 fields, saw 14



["It's amazing..... battery last around 10 -12 hrs.....best laptop for students and coders",
 "Finally a MacBook possession. Got it in 50k in Diwali sale. Technically good laptop.Good News.luxury product, niche technology, good battery life, reliable machine (other laptops crash and become defunct in 6-10 years), no anti-virus required, security of data and transactions much better. Terrific sense of possession & pride.Bad News. Problems will be there if you are switching from windows based system - very less space in hard disk, no CD drive, inability to transfer data from Mac to your existing external hard disc unless you format it, apps are mostly paid and re unreasonably expensive (no free apps which are available otherwise on google play store, even the angry bird costs Rs 400 !!), all printers are not compatible (e.g. the most economical MFD laser printer RICOH SP 111 can't be used), huge compatibility issues with Pages (MS Word) and Keynote (powerpoint) unless you master it by wo

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk import tokenize
sentences=tokenize.sent_tokenize(" ".join(book))
sentences[5:15]

['!',
 '), all printers are not compatible (e.g.',
 "the most economical MFD laser printer RICOH SP 111 can't be used), huge compatibility issues with Pages (MS Word) and Keynote (powerpoint) unless you master it by working on these (still mostly the document and slides either do not open in windows environment or have distortion issues).",
 'you need to spend extra for an external CD writer and tp buy a carry bag.',
 'Mac con not be connected to most of the projectors unless you buy some connectors, which are expensive and theres no clarity which one to buy and from where.',
 "you can't connect it to your TV, the ports are different and again theres no clarity - customer support, manuals or help section are silent !",
 'Marketing strategy for Indian market, if at all has been planned, has been very bad.',
 'If these critical issues (and a few more not mentioned here) are handled properly, the Mac can sweep away other laptops in India.',
 "Presently, it doesn't seem to be happening.Ove

In [ ]:
sent_df=pd.DataFrame(sentences,columns=["sentence"])
sent_df

,sentence
0,It's amazing..... battery last around 10 -12 h...
1,Got it in 50k in Diwali sale.
2,Technically good laptop.Good News.luxury produ...
3,Terrific sense of possession & pride.Bad News.
4,Problems will be there if you are switching fr...
...,...
183,You can get one of the best movie experiences ...
184,Overall I would it a 4.5/5.
185,"I got a great deal on this laptop, which I hav..."
186,I got it not only on discount but also was abl...


In [ ]:
affinity_scores=afinn.set_index("word")["value"].to_dict()

In [ ]:
affinity_scores

In [ ]:
nlp=spacy.load("en_core_web_sm")
sentiment_lexicon=affinity_scores

def calculate_sentiment(text: str=None):
  sent_score=0
  if text:
    sentence=nlp(text)
    print(sentence)
    for word in sentence:
      sent_score +=sentiment_lexicon.get(word.lemma_,0)
  return sent_score



In [ ]:
#Test that the for loop works
calculate_sentiment(text="unhappy")

unhappy


-2

In [ ]:
calculate_sentiment(text="amazing")

amazing


4

In [ ]:
sent_df["sentiment_value"]=sent_df["sentence"].apply(calculate_sentiment)

In [ ]:
sent_df

,sentence,sentiment_value
0,It's amazing..... battery last around 10 -12 h...,7
1,Got it in 50k in Diwali sale.,0
2,Technically good laptop.Good News.luxury produ...,5
3,Terrific sense of possession & pride.Bad News.,4
4,Problems will be there if you are switching fr...,-11
...,...,...
183,You can get one of the best movie experiences ...,3
184,Overall I would it a 4.5/5.,0
185,"I got a great deal on this laptop, which I hav...",3
186,I got it not only on discount but also was abl...,0


In [ ]:
#How many words are there in the sentence?
sent_df["word_count"]=sent_df["sentence"].str.split().apply(len)
sent_df["word_count"].head(10)

0    17
1     7
2    31
3     7
4    65
5     1
6     7
7    47
8    16
9    29
Name: word_count, dtype: int64

In [ ]:
sent_df.sort_values(by="sentiment_value").tail(10)

,sentence,sentiment_value,word_count
111,"The build is super strong, the operating syste...",5,10
46,Now my mac is perfectly fine : Got the Apple M...,5,27
47,Ships with macOS sierra but update is availabl...,6,86
65,I hope this comment can help to save somebody’...,6,10
104,The switch was inspired by the awesome reviews...,6,13
60,I use this for music production and it works a...,7,30
179,MERITS..IT IS BEST IN ITS PERFORMANCE..SOUND Q...,7,43
0,It's amazing..... battery last around 10 -12 h...,7,17
152,..So now I m working with the external keyboar...,17,113
87,"All are Gud.. For now.. Apple,apple,apple, tha...",24,184


In [ ]:
#Sentiment score of the whole rewiew
sent_df["sentiment_value"].describe()

count    188.000000
mean       0.930851
std        3.379793
min      -11.000000
25%        0.000000
50%        0.000000
75%        3.000000
max       24.000000
Name: sentiment_value, dtype: float64

In [ ]:
sent_df[sent_df["sentiment_value"]<=0].head()

,sentence,sentiment_value,word_count
1,Got it in 50k in Diwali sale.,0,7
4,Problems will be there if you are switching fr...,-11,65
5,!,0,1
6,"), all printers are not compatible (e.g.",0,7
8,you need to spend extra for an external CD wri...,0,16
